In [11]:
from os import listdir
import scipy.interpolate as sp
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seawater as sw
from tqdm import tqdm
from scipy.interpolate import griddata

In [12]:
path="/tigress/GEOCLIM/LRGROUP/shared_data/woa18/WOA18_0to1500m/"
lonmin,lonmax=35,130
latmin,latmax=-30,30
for tt in tqdm(range(1,13)):
    
    if tt == 1:
        TEMP = np.zeros((12,57,60,95))
        DOXY = np.zeros((12,57,60,95))
        PSAL = np.zeros((12,57,60,95))
        
    if tt < 10:
        t='0'+str(tt)
    else:
        t=str(tt)
        
    # TEMP
    data=xr.open_dataset(path+"temperature/woa18_decav_t"+t+"_01.nc", decode_times=False)
    tmp=data["t_an"][0,:,:,:]
    Lat_s=np.logical_and(tmp.lat>latmin,tmp.lat<latmax)
    Lon_s=np.logical_and(tmp.lon>lonmin,tmp.lon<lonmax)
    tmp=np.array(tmp)[:,Lat_s]
    tmp=np.array(tmp)[:,:,Lon_s]
    TEMP[tt-1,:,:,:] = tmp
    
    depth_1500=data.depth
    
    # PSAL
    data=xr.open_dataset(path+"salinity/woa18_decav_s"+t+"_01.nc", decode_times=False)
    tmp=data["s_an"][0,:,:,:]
    Lat_s=np.logical_and(tmp.lat>latmin,tmp.lat<latmax)
    Lon_s=np.logical_and(tmp.lon>lonmin,tmp.lon<lonmax)
    tmp=np.array(tmp)[:,Lat_s]
    tmp=np.array(tmp)[:,:,Lon_s]
    PSAL[tt-1,:,:,:] = tmp
    
    # DOXY
    data=xr.open_dataset(path+"oxygen/woa18_all_o"+t+"_01.nc", decode_times=False)
    tmp=data["o_an"][0,:,:,:]
    Lat_s=np.logical_and(tmp.lat>latmin,tmp.lat<latmax)
    Lon_s=np.logical_and(tmp.lon>lonmin,tmp.lon<lonmax)
    tmp=np.array(tmp)[:,Lat_s]
    tmp=np.array(tmp)[:,:,Lon_s]
    DOXY[tt-1,:,:,:] = tmp
    

LON=np.array(data.lon)[Lon_s]
LAT=np.array(data.lat)[Lat_s]
DEPTH=np.array(depth_1500)

100%|██████████| 12/12 [00:01<00:00,  5.89it/s]


# Interpolation

In [61]:
interp_min=0 # min pressure (in dbar)
interp_max=1000 # max pressure (in dbar)
interp_step=5

PRES_i=np.linspace(interp_min,interp_max,1+int((interp_max-interp_min)/interp_step))

n = len(LAT)
m = len(LON)
t = 12
with tqdm(total=n * m * t) as pbar:
    for tt in range(12):
        for ii in range(len(LAT)):
            for jj in range(len(LON)):

                if tt == 0:
                    TEMP_i = np.zeros((12,201,60,95))*np.nan
                    DOXY_i = np.zeros((12,201,60,95))*np.nan
                    PSAL_i = np.zeros((12,201,60,95))*np.nan

                # conversion [m] -> [dbar]:
                pres=sw.eos80.pres(DEPTH, [lat for i in range(len(DEPTH))])

                # temp,psal,doxy selection
                temp,psal,doxy = TEMP[tt,:,ii,jj],PSAL[tt,:,ii,jj],DOXY[tt,:,ii,jj]

                # interpolation
                ft = sp.interp1d(pres,temp,bounds_error=False,fill_value=np.nan)
                TEMP_i[tt,:,ii,jj] = ft(PRES_i)
                fo = sp.interp1d(pres,doxy,bounds_error=False,fill_value=np.nan)
                DOXY_i[tt,:,ii,jj] = fo(PRES_i)
                fs = sp.interp1d(pres,psal,bounds_error=False,fill_value=np.nan)
                PSAL_i[tt,:,ii,jj] = fs(PRES_i)
                
                pbar.update(1)
            

            
ds_i=xr.Dataset(coords={'time': np.arange(1,13),
                        'pres': PRES_i,
                        'lon': LON,
                        'lat': LAT})
# add to dataset
ds_i['TEMP_grid'] = xr.DataArray(TEMP_i,dims = ['time','pres','lat','lon'],coords =[np.arange(1,13),PRES_i,LAT,LON])
ds_i['PSAL_grid'] = xr.DataArray(PSAL_i,dims = ['time','pres','lat','lon'],coords =[np.arange(1,13),PRES_i,LAT,LON])
ds_i['DOXY_grid'] = xr.DataArray(DOXY_i,dims = ['time','pres','lat','lon'],coords =[np.arange(1,13),PRES_i,LAT,LON])
ds_i          

<xarray.Dataset>
Dimensions:    (lat: 60, lon: 95, pres: 201, time: 12)
Coordinates:
  * time       (time) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * pres       (pres) float64 0.0 5.0 10.0 15.0 20.0 ... 985.0 990.0 995.0 1e+03
  * lon        (lon) float32 35.5 36.5 37.5 38.5 ... 126.5 127.5 128.5 129.5
  * lat        (lat) float32 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
Data variables:
    TEMP_grid  (time, pres, lat, lon) float64 nan nan nan nan ... nan 4.5 nan
    PSAL_grid  (time, pres, lat, lon) float64 nan nan nan nan ... nan 34.44 nan
    DOXY_grid  (time, pres, lat, lon) float64 nan nan nan nan ... nan 75.65 nan

In [48]:
# get merged data from tigress
path_AWG = '../Merged/'
infn_AWG = 'Profiles_temp_psal_doxy_processed.nc'

ds_AWG = xr.open_dataset(path_AWG + infn_AWG)
ds_AWG

<xarray.Dataset>
Dimensions:  (pres: 201, time: 306118)
Coordinates:
    prof     (time) int32 ...
  * time     (time) datetime64[ns] 1827-10-30 1836-05-15 ... 2020-02-07
  * pres     (pres) int32 0 5 10 15 20 25 30 35 ... 970 975 980 985 990 995 1000
Data variables:
    temp     (time, pres) float32 ...
    psal     (time, pres) float32 ...
    doxy     (time, pres) float32 ...
    lat      (time) float32 ...
    lon      (time) float32 ...
    type     (time) int32 ...

In [ ]:
xx,yy = np.meshgrid(ds_i.lon,ds_i.lat)
xx = xx.flatten()
yy = yy.flatten()

lons= np.array(ds_AWG.lon)
lats = np.array(ds_AWG.lat)
months = np.array(ds_AWG.time.dt.month)

TEMP_AWG = np.zeros((len(months),len(PRES_i)))*np.nan
DOXY_AWG = np.zeros((len(months),len(PRES_i)))*np.nan
PSAL_AWG = np.zeros((len(months),len(PRES_i)))*np.nan
lon_AWG = np.zeros(len(months))*np.nan
lat_AWG = np.zeros(len(months))*np.nan
month_AWG = np.array(months)

n = len(ds_i.time)
m = len(PRES_i)
with tqdm(total=n * m) as pbar:
    for tt in range(n):
        for dd in range(m):

            temp = np.array(ds_i.TEMP_grid[tt,dd,:,:])
            psal = np.array(ds_i.PSAL_grid[tt,dd,:,:])
            doxy = np.array(ds_i.DOXY_grid[tt,dd,:,:])
            indq = months == np.array(ds_i.time[tt])   
            lonq = lons[indq]
            latq = lats[indq]

            # find the lons and lats of TCD vals at this time

            points = np.array( (xx,yy) ).T
            temp_values = temp.flatten()
            psal_values = psal.flatten()
            doxy_values = doxy.flatten()

            TEMP_AWG[indq,dd] = griddata(points, temp_values, (lonq,latq) ,method='linear')
            PSAL_AWG[indq,dd] = griddata(points, psal_values, (lonq,latq) ,method='linear')
            DOXY_AWG[indq,dd] = griddata(points, doxy_values, (lonq,latq) ,method='linear')
            lon_AWG[indq] = lonq
            lat_AWG[indq] = latq
            pbar.update(1)

 73%|███████▎  | 1750/2412 [07:44<02:53,  3.82it/s]

# SAVE

In [ ]:
ds=xr.Dataset(coords={'month': month_AWG,
                        'pres': PRES_i})
# add to dataset
ds['temp_AWG'] = xr.DataArray(TEMP_AWG,dims = ['month','pres'],coords =[month_AWG,PRES_i])
ds['psal_AWG'] = xr.DataArray(PSAL_AWG,dims = ['month','pres'],coords =[month_AWG,PRES_i])
ds['doxy_AWG'] = xr.DataArray(DOXY_AWG,dims = ['month','pres'],coords =[month_AWG,PRES_i])
ds['lon_AWG'] = xr.DataArray(lon_AWG,dims = ['month'],coords =[month_AWG])
ds['lat_AWG'] = xr.DataArray(lat_AWG,dims = ['month'],coords =[month_AWG])

# save_path = '/projects/GEOCLIM/LRGROUP/jennap/Modulation_of_Coastal_Hypoxia_by_the_IOD/data/'
save_path = ""
ds.to_netcdf('woa_awg_processed.nc',mode='w',format = "NETCDF4")
ds